V nalogi bomo delali s pogostostmi priimkov v slovenskih občinah. Podatki so zbrani okrog leta 2000 in za vsako občino vsebujejo število gospodinjstev z enim od 200 najpogostejših slovenskih priimkov. Zaradi anonimizacije je nekaj gospodinjstev v bližini občinske meje štetih v napačno občino. Možne so tudi druge manjše napake, ki izvirajo iz načina zbiranja podatkov.

Delali bomo z dvema datotekama v enaki obliki. Razlikujeta se po številu občin in priimkov. 

- `priimki-v-obcinah-mali.csv` vsebuje podatke za 5 občin in 4 priimke.
- `priimki-v-obcinah.csv` vsebuje podatke za vseh 192 občin in 200 priimkov.

Datoteki naj bosta v istem direktoriju kot vaš program.

V spodnjih primerih bomo uporabljali manjšo datoteko. Vaše funkcije morajo pravilno delovati z obema datotekama in tudi morebitnimi drugimi datotekami v enaki obliki (torej: z drugačnim številom občin in priimkov).

Oblika datotek je naslednja:

```csv
Občina,Površina [km²],Število prebivalcev,NOVAK,HORVAT,KOVAČIČ,KRAJNC
Ljubljana,275.0,276091,479,245,189,58
Maribor,147.5,113113,186,204,104,258
Tolmin,381.5,10953,3,0,54,1
Celje,94.9,48679,50,39,49,104
Murska Sobota,64.4,19433,73,245,16,0
```

Prva vrstica vsebuje imena stolpcev. Prvi trije stolpci vsebujejo podatke o občini, ostali pa število gospodinjstev s posameznim priimkom v tej občini. (Večja datoteka vsebuje enake tri stolpce, nato pa 200 stolpcev za priimke.)

Nekatere naloge morate rešiti z eno vrstico numpyja -- ker se da in ker morate znati. Pri ostalih počnite, kar želite. Dodatnih funkcij ni dovoljeno pisati (da ne bi "enovrstičnih" prenesli v druge funkcije :).

### 1. Branje podatkov

Napišite funkcijo `preberi(datoteka)`, ki prebere datoteko in vrne terko z naslednjimi tabelami:

- imena občin (kot nizi)
- površine občin (kot števila s plavajočo vejico, float)
- števila prebivalcev občin (kot cela števila, int)
- pogostosti priimkov (kot dvodimenzionalna tabela celih števil, int)
- imena priimkov (kot nizi)

Klic `preberi("priimki-v-obcinah-mali.csv")` naj vrne:

```
(np.array(['Ljubljana', 'Maribor', 'Tolmin', 'Celje', 'Murska Sobota']),
 np.array([275.0, 147.5, 381.5, 94.9, 64.4]),
 np.array([276091, 113113, 10953, 48679, 19433]),
 np.array([[479, 245, 189, 58],
           [186, 204, 104, 258],
           [3, 0, 54, 1],
           [50, 39, 49, 104],
           [73, 245, 16, 0]],
 np.array(['NOVAK', 'HORVAT', 'KOVAČIČ', 'KRAJNC']))
```

**Pomoč**: `genfromtxt` ima argument `skip_header`. Uporabite `skip_header=1`, da preskočite prvo vrstico. Le-to lahko preberete ločeno, tako kot običajno beremo datoteke v Pythonu.

#### Rešitev

In [3]:
import numpy as np

def preberi(ime_datoteke):
    priimki = np.array(open(ime_datoteke, encoding="utf-8").readline().strip().split(",")[3:])
    podatki = np.genfromtxt(ime_datoteke, encoding="utf-8", delimiter=",", dtype=str, skip_header=1)
    obcine = podatki[:, 0]
    povrsina = podatki[:, 1].astype(float)
    prebivalci = podatki[:, 2].astype(int)
    pogostosti = podatki[:, 3:].astype(int)
    return obcine, povrsina, prebivalci, pogostosti, priimki

S priimki smo opravili na hitro: odprli datoteko (`open(ime_datoteke, encoding="utf-8")`), prebrali prvo vrstico (`.readline()`), iz dobljenega niza odbili `\n` na koncu (`.strip()`), ga razbili glede na vejico (`.split(",")`) in odbili prve tri reči (`[3:]`). Ena vrstica, pa je urejeno.

Nato se lotimo podatkov. Podatke preberemo z `genfromtxt` z ustreznii argumenti: nastaviti je potrebno `encoding` (hvala, Microsoft), določiti ločilo (`delimiter`) in podatkovni tip (`dtype`) ter preskočiti prvo vrstico (`skip_header`).

Sledi le še vaja iz indeksiranja dvodimenzionalnih tabel in pretvarjanja tipov. Vedno pobiramo vse vrstice (zato `:` kot prva dimenzija indeksa), potem pa vzamemo ničti, prvi, drugi stolpec, ali pa vse stolpce razen prvih treh. Tabele pretvorimo v `int` ali `float`, kakor veleva naloga.

In [4]:
obcine, povrsina, prebivalci, pogostosti, priimki = preberi("priimki-v-obcinah-mali.csv")

In [5]:
obcine

array(['Ljubljana', 'Maribor', 'Tolmin', 'Celje', 'Murska Sobota'],
      dtype='<U13')

In [6]:
povrsina

array([275. , 147.5, 381.5,  94.9,  64.4])

In [7]:
prebivalci

array([276091, 113113,  10953,  48679,  19433])

In [8]:
pogostosti

array([[479, 245, 189,  58],
       [186, 204, 104, 258],
       [  3,   0,  54,   1],
       [ 50,  39,  49, 104],
       [ 73, 245,  16,   0]])

In [9]:
priimki

array(['NOVAK', 'HORVAT', 'KOVAČIČ', 'KRAJNC'], dtype='<U7')

### 2. Najpogostejši priimek

Napišite funkcijo `najpogostejsi_priimek(pogostosti, priimki)`, ki vrne najpogostejši priimek (glede na število gospodinjstev) v vseh občinah skupaj.

Če funkcija dobi vse podatke, vrne "NOVAK", saj ima ta priimek največ gospodinjstev. Tudi če jo poženemo na malih podatkih, vrne Novak (v podatkih je 791 Novakov in "samo" 733 Horvatov). Če pa jo poženemo na malih podatkih brez prve vrstice (Ljubljana), vrne "HORVAT".

Funkcija naj bo narejena z eno vrstico numpyja.

#### Rešitev

Naloga pravi "v vseh občinah skupaj", torej moramo za začetek sešteti pogostosti priimkov prek vseh občin, torej prek osi 0.

In [11]:
np.sum(pogostosti, axis=0)

array([791, 733, 412, 421])

Pogostosti, 791, 733 in tako naprej se nanašajo na priimke v tabeli `priimki`.

In [12]:
priimki

array(['NOVAK', 'HORVAT', 'KOVAČIČ', 'KRAJNC'], dtype='<U7')

Vrniti moramo priimek, ki se nanaša na največjo število v gornji vsoti. Spet se izkaže za koristnega `argmax`: vrne indeks največjega elementa v `pogostosti` in vrnemo element s tem indeksom iz `priimki`.

In [14]:
np.argmax(np.sum([791, 733, 412, 421], axis=0))

np.int64(0)

In [15]:
priimki[np.argmax(np.sum([791, 733, 412, 421], axis=0))]

np.str_('NOVAK')

Rešitev naloge je torej funkcija

In [16]:
def najpogostejsi_priimek(pogostosti, priimki):
    return priimki[np.argmax(np.sum(pogostosti, axis=0))]

### 3. Deleži po občinah

Napišite funkcijo `delezi_po_obcinah(pogostosti)`, ki vrne dvodimenzionalno tabelo z enakimi dimenzijami kot `pogostosti`, kjer je v vsaki vrstici delež gospodinjstev s tem priimkom v Sloveniji.

Funkcija naj bo narejena z eno vrstico numpyja.

Če funkcijo poženemo na malih podatkih, mora vrniti:

```
array([[0.60556258, 0.33424284, 0.45873786, 0.13776722],
       [0.23514539, 0.27830832, 0.25242718, 0.6128266 ],
       [0.00379267, 0.        , 0.13106796, 0.0023753 ],
       [0.06321113, 0.053206  , 0.11893204, 0.24703088],
       [0.09228824, 0.33424284, 0.03883495, 0.        ]])
```

Prvi stolpec pove, kako so porazdeljeni Novaki po občinah, drugi stolpec Horvati itd. Če se spomnimo, kako gredo kraji po vrsti (Lj, Mb, Tm, Ce, Ms), lahko na primer vidimo, da je 60.56 % vseh Novakov v Ljubljani, 23.51 % v Mariboru itd.


**Pomoč:** k dvodimenzionalni tabeli lahko prištejemo, odštejemo, množimo, delimo ... enodimenzionalno tabelo, ki ima enako elementov kot vrstica dvodimenzionalne tabele. Npr. če imamo tabelo `a` oblike (3, 4) in tabelo `b` oblike (4,), potem izraz `a / b` pomeni, da se vsak stolpec tabele `a` deli z ustreznim elementom tabele `b`. Tule je primer za množenje:

```
>>> a = np.array(
        [[2, 6, 8, 1],
         [1, 2, 3, 4],
         [3, 8, 3, 1]])
>>> f = np.array([10, 50, 100, 1000])
>>> a * f
array([[  20,  300,  800, 1000],
       [  10,  100,  300, 4000],
       [  30,  400,  300, 1000]])
```

**Spoiler**: vsak stolpec delite z vsoto stolpca.

#### Rešitev

V prejšnji nalogi smo že računali vsote stolpcev. Zdaj jih uporabimo za to, da z njimi delimo tabelo. Gornji primer je pokazal množenje; tu bomo pač delili.

In [17]:
def delezi_po_obcinah(pogostosti):
    return pogostosti / np.sum(pogostosti, axis=0)

### 4. Specifični priimki

Poglejmo rezultat prejšnje naloge. Za večjo preglednost bom spremenil podatke v odstotke in jih zaokrožil na dve decimalni mesti (vam tega ni potrebno početi, to počnem le za razlago!).

	```
	              Novak   Horvat  Kovačič   Krajnc
	Ljubljana     60.56    33.42    45.87    13.78
	Maribor       23.51    27.83    25.24    61.28
	Tolmin         0.38     0.00    13.11     0.24
	Celje          6.32     5.32    11.89    24.70
	Murska Sobota  9.23    33.42     3.88     0.00
	```

    Novak je gotovo najpogostejši priimek v Ljubljani. Za Maribor je mogoče bolj specifičen Krajnc, saj v Mariboru živi 61 % vseh slovenskih Krajncev (in le 23 % vseh Novakov). Za Tolmin je torej najbolj specifičen priimek Kovačič, saj je tam kar 13 % vseh Kovačičev v Sloveniji v Tolminu. Za Celje je najbolj specifičen Krajnc, za Mursko Soboto pa seveda Horvat. (**To so seveda mali podatki,** ki vključujejo le teh pet občin. V resnici v Ljubljani ne živi 60 % vseh Novakov!)
	
	Napišite funkcijo `specificni_priimki(pogostosti, priimki)`, ki prejme tabelo pogostosti (takšno, kot je preberemo iz datoteke) in vrne tabelo z najbolj specifičnimi priimki po občinah. Pri tem naj pokliče `delezi_po_obcinah`, da bodo pogostosti priimkov normalizirane.

	Funkcija naj bo narejena z eno vrstico numpyja.

	Za male podatke funkcija vrne `np.array(['NOVAK', 'KRAJNC', 'KOVAČIČ', 'KRAJNC', 'HORVAT'])`.

	Torej: v Ljubljani je najbolj specifičen priimek Novak, v Mariboru Krajnc, v Tolminu Kovačič, v Celju Krajnc in v Murski Soboti Horvat.

#### Rešitev

Tule očitno nadaljujemo iz tretje naloge. Za vsako vrstico (občina) moramo izvedeti najpogostejši priimek. Tako kot v drugi nalogi uporabimo `argmax`, da dobimo indeks, s katerim bomo hodili gledat v tabelo `priimki`. Razlika je v tem, da smo imeli pri drugi nalogi enodimenzionalno tabelo (pogostosti priimkov v vsej Sloveniji) tu pa dvodimenzionalno. `maxarg` moramo iskati prek stolpcev - zanima nas stolpec z največjim elementom (za vsako vrstico, občino). Torej:

In [18]:
np.argmax(delezi_po_obcinah(pogostosti), axis=1)

array([0, 3, 2, 3, 1])

V prvi (no, ničti...) vrstici je največji element v ničtem stolpcu, v drugi vrstici v tretjem, in tako naprej. Zdaj moramo iz tabele `priimki` pobrati elemente s temi indeksi.

In [19]:
priimki[[0, 3, 2, 3, 1]]

array(['NOVAK', 'KRAJNC', 'KOVAČIČ', 'KRAJNC', 'HORVAT'], dtype='<U7')

Oziroma, v splošnem

In [20]:
priimki[np.argmax(delezi_po_obcinah(pogostosti), axis=1)]

array(['NOVAK', 'KRAJNC', 'KOVAČIČ', 'KRAJNC', 'HORVAT'], dtype='<U7')

Rešitev je torej funkcija

In [21]:
def specificni_priimki(pogostosti, priimki):
    return priimki[np.argmax(delezi_po_obcinah(pogostosti), axis=1)]

### 5-6. Izpis specifičnih priimkov

Napiši funkcijo `izpis_specificnih(obcine, pogostosti, priimki)`, ki pokliče funkcijo iz prejšnje naloge, da izve najbolj specifične priimke po občinah, nato pa izpiše rezultate v obliki:

```
                           Ljubljana - NOVAK
                             Maribor - KRAJNC
                              Tolmin - KOVAČIČ
                               Celje - KRAJNC
                       Murska Sobota - HORVAT
```

Ime občine je izpisano na 40 mest in poravnano desno, sledi presledek, minus, presledek, nato pa najbolj specifičen priimek za to občino.

Testi bodo izpisali najbolj specifične priimke za vse občine v večji datoteki. Kdor o tem kaj ve (na primer prihaja iz drugega konca Slovenije, ima tam sorodnike), naj si ga ogleda, saj je kar zanimiv.

#### Rešitev

Tole ni vaja iz numpyja temveč samo iz `zip`-a in oblikovanja nizov.

In [22]:
def izpis_specificnih(obcine, pogostosti, priimki):
    for obcina, priimek in zip(obcine, specificni_priimki(pogostosti, priimki)):
        print(f"{obcina:>40} - {priimek}")

### 7-8. Občine s specifičnimi priimki

Napišite funkcijo `izpis_obcin_s_priimki(obcine, pogostosti, priimki)`, ki najprej poišče najbolj specifične priimke za posamične občine (torej: pokličite `specificni_priimki`). Nato izpiše vse priimke, ki se pojavijo med najbolj specifičnimi, za vsakim priimkom pa dvopičje in nato vse občine, za katere je ta priimek specifičen. Tako priimki kot občine morajo biti urejeni po abecedi (pri čemer so Č, Š in Ž na koncu, za črko Z, saj se nismo naučili, kako Python prisiliti, da pravilno ureja po slovenski abecedi).

Če želite reševati nalogo, kot se spodobi za `numpy`, vam bo prišel prav `unique`. Druga možnost je, da uporabite običajne Pythonove slovarje.

Izpis za male podatke bo videti takole:

```
HORVAT: Murska Sobota
KOVAČIČ: Tolmin
KRAJNC: Celje, Maribor
NOVAK: Ljubljana
```

Izpis za celotne podatke se začne z:

```
AMBROŽIČ: Bled
BAJC: Ajdovščina, Postojna
BERGANT: Komenda, Tabor
BEVC: Bistrica ob Sotli
BEZJAK: Gorišnica, Markovci
BLAŽIČ: Miren - Kostanjevica
BOGATAJ: Gorenja vas - Poljane, Žiri
BREGAR: Radeče, Trbovlje
BREZNIK: Luče, Mežica, Muta
BUKOVEC: Kobilje
BUČAR: Dolenjske Toplice, Šmartno pri Litiji
CERAR: Domžale, Lukovica, Moravče
DOLINAR: Dobrova - Polhov Gradec
```

in konča z:

```
ZUPAN: Žirovnica
ZVER: Beltinci, Odranci, Turnišče
ČEH: Ptuj
ŠTRUKELJ: Nazarje
ŠULIGOJ: Kanal, Nova Gorica
ŠUŠTAR: Kamnik
ŠUŠTERŠIČ: Brezovica, Ljubljana
ŽAGAR: Kostel
ŽELEZNIK: Žetale
ŽIBERT: Sevnica
```
#### Rešitev

Tole je za večino naloga iz slovarjev. Rešimo jo tako:

In [24]:
def izpis_obcin_s_priimki(obcine, pogostosti, priimki):
    specificni = {}
    for obcina, priimek in zip(obcine, specificni_priimki(pogostosti, priimki)):
        if priimek not in specificni:
            specificni[priimek] = []
        specificni[priimek].append(obcina)

    for priimek, obcine in sorted(specificni.items()):
        print(f"{priimek}: {', '.join(sorted(obcine))}")

`specificni` je slovar, katerega ključi bodo priimki, vrednosti pa seznam občin, za katere je specifičen ta priimek.

Zanko zapodimo čez enak zip kot v prejšnji nalogi, le da priimkov in občin ne izpisujemo, temveč jih dodajamo v slovar (`specificni[priimek].append(obcina)`). Še prej pa preverimo, ali ključ že obstaja; če ga ni, ga dodamo.

V drugem delu gre zanka čez slovar `specificni`, točneje, čez pare njegovih ključev in vrednosti. Funkcija `sort` bo uredila terke najprej po prvem elementu, če bi imeli dve terki enak prvi element, pa po drugem. To se tu ne more zgoditi, saj gre za slovar in so vsi prvi elementi terk seveda unikatni. S `print` izpišemo priimek in nato urejen seznam občin, ki jih združimo z `", ".join`.

Namesto običajnega slovarja lahko uporabimo slovar s privzetimi vrednostmi.

In [25]:
from collections import defaultdict

def izpis_obcin_s_priimki(obcine, pogostosti, priimki):
    specificni = defaultdict(list)
    for obcina, priimek in zip(obcine, specificni_priimki(pogostosti, priimki)):
        specificni[priimek].append(obcina)

    for priimek, obcine in sorted(specificni.items()):
        print(f"{priimek}: {', '.join(sorted(obcine))}")

Če želimo nalogo reševati z numpyjem, uporabimo `np.unique`. Funkcija `np.unique(arr)` prejme tabelo in vrne novo tabelo, v kateri se vsak element prvotne pojavi le enkrat. Nova tabela je, mimogrede, tudi urejena.

In [26]:
spec_priimki = specificni_priimki(pogostosti, priimki)

In [27]:
np.unique(spec_priimki)

array(['HORVAT', 'KOVAČIČ', 'KRAJNC', 'NOVAK'], dtype='<U7')

To ni še nič tako posebnega, podobno reč (le nujno neurejeno) bi dobili tudi z množicami.

In [28]:
set(spec_priimki)

{np.str_('HORVAT'), np.str_('KOVAČIČ'), np.str_('KRAJNC'), np.str_('NOVAK')}

`np.unique` ima še nekaj imenitnih dodatnih argumentov; eden od njih je `return_inverse`, ki ga lahko nastavimo na `True`.

In [29]:
np.unique(spec_priimki, return_inverse=True)

(array(['HORVAT', 'KOVAČIČ', 'KRAJNC', 'NOVAK'], dtype='<U7'),
 array([3, 2, 1, 2, 0]))

Če `np.unique` pokličemo z `return_inverse=True`, vrne terko z dvema rečema. Prva je tabela, kot prej. Drugi je tabela, ki je dolga toliko kot tabela, ki smo jo podali funkciji (v našem primeru `spec_priimki`) in vsebuje indekse v vrnjeno tabelo unikatnih elementov. Spomnimo se, kako so videti `spec_priimki`:

In [32]:
spec_priimki

array(['NOVAK', 'KRAJNC', 'KOVAČIČ', 'KRAJNC', 'HORVAT'], dtype='<U7')

Prvi priimek iz `priimki` (NOVAK) se v tabelih unikatnih elementov nahaja na indeksu 3. Drugi priimek (KRAJNC) je na indeksu 2.

Tule je mogoče potrebno malo zakravžljati možgane. Pokažimo v živo.

In [34]:
unikatni, indeksi = np.unique(spec_priimki, return_inverse=True)

In [35]:
unikatni

array(['HORVAT', 'KOVAČIČ', 'KRAJNC', 'NOVAK'], dtype='<U7')

In [36]:
indeksi

array([3, 2, 1, 2, 0])

In [37]:
spec_priimki

array(['NOVAK', 'KRAJNC', 'KOVAČIČ', 'KRAJNC', 'HORVAT'], dtype='<U7')

In [38]:
unikatni[indeksi[0]]

np.str_('NOVAK')

In [39]:
spec_priimki[0]

np.str_('NOVAK')

In [40]:
unikatni[indeksi[1]]

np.str_('KRAJNC')

In [41]:
spec_priimki[1]

np.str_('KRAJNC')

In [42]:
for i in range(len(indeksi)):
    print(unikatni[indeksi[i]], spec_priimki[i])

NOVAK NOVAK
KRAJNC KRAJNC
KOVAČIČ KOVAČIČ
KRAJNC KRAJNC
HORVAT HORVAT


Ali, na hitro

In [43]:
unikatni[indeksi]

array(['NOVAK', 'KRAJNC', 'KOVAČIČ', 'KRAJNC', 'HORVAT'], dtype='<U7')

In [44]:
unikatni[indeksi] == spec_priimki

array([ True,  True,  True,  True,  True])

Če uporabimo `np.unique`, se znebimo prve zanke v naši funkciji:

In [45]:
def izpis_obcin_s_priimki(obcine, pogostosti, priimki):
    priimki = specificni_priimki(pogostosti, priimki)
    unikatni, indeksi = np.unique(priimki, return_inverse=True)
    for i, priimek in enumerate(unikatni):
        obcine_s_tem_priimkom = obcine[indeksi == i]
        print(f"{priimek}: {', '.join(sorted(obcine_s_tem_priimkom))}")

Kaj sta `unikatni` in `indeksi` smo videli zgoraj.

Ponovimo le, kaj nam v bistvo pove `indeksi`: dolga je toliko kot `spec_priimki` - vsebuje torej en indeks za vsako občino. Če je `indeksi` v j-ti vrstici enak `i`, to pomeni, da je za `j`-to občino specifičen `i`-ti priimek (v tabeli `unikatni`).

Druga zanka, ki skrbi za izpis, mora narediti naslednje. Gre čez unikatne priimke in njihove zaporedne številke. `print` bo izpisal `i`-ti priimek, dobiti pa mora občine, za katere je ta priimek specifičen. To pa so tiste občine, pri katerih je v `indeksi` element `i`. (Če ne razumete, ponovno preberite prejšnji odstavek.) Zanimajo nas imena teh občin, torej `obcine[indeksi == i]`. Posortiramo, združimo, izpišemo.